# KZG Implementation(Python Version)

The KZG Commitment Scheme is a commitment scheme that allows to commit to a polynomial $\Phi(x) = \phi_0 +\phi_1x+\phi_2x^2+...+\phi_lx^l$, where $\Phi(x) \in F_p[x]$ . 'to commit' means proving that you know the polynomial $\Phi(x)$ without revealing it.

The KZG commitment scheme consists of 4 steps:
1. Setup
2. Commit to Polynomials
3. Prove an Evaluation
4. Verify an Evaluation Proof

In [1]:
from sage import *

## Step 1: Setup

The first step is an one-time trusted setup and once it has done once, the following steps can be done repeatedly
1. Let $G$ be a pairing-friendly elliptic curve group
2. Let $g$ be a generator of $G$
3. Let $l$ be the maximum degree of the polymonials we want to commit to ($l < p$)
4. Pick some random field element as secret parameter $\tau \in F_p$(usually done by MPC, to simplify, we just randomly choose one here)
5. Compute pp(public parameters)$$(g, g^\tau, g^{\tau^2},...,g^{\tau^l})$$ and release it publicly
6. Discard secret parameter $\tau$ once the setup ceremony is done so that nobody can figure out its value

In [15]:
# generate a ramdom prime p with k bits of security
def safe_random_prime(k):
    '''generate a random prime p
    
    usually when p = 2q + 1, for p,q are all prime, then p is a secure prime which can reach the maxium security
    
    Args:
        k: k bits of security that we want to achieve in the commitment scheme
    '''
    while true:
        p = random_prime(2^k-1, false, 2^(k-1))
        if ZZ((p-1)/2).is_prime():
            return p
        
# usually we need k to be at least 128 bits, but we use 16 here just for demo  
k = 16
p = safe_random_prime(k)
print(f'Our safe random prime is {p}')

Our safe random prime is 53267


In [16]:
# define galois field
F_p = GF(p)

# 1. Let G be a pairing-friendly elliptic curve group
    # to simplify, the elliptic curve group G is also a galois field here, just because it is cyclic and has a generator g
    # but it is less secure for sure, we will replace it in the subsequent project
G = F_p

# 2. Let g be a generator of G
g = G.multiplicative_generator()
print(f'generator g is {g}')

# 3. Let l be the maximum degree of the polymonials we want to commit to, where l < p
    # randomly generate here
l = 0
while l == 0:
    l = F_p.random_element()
print(f'the maximum degree l of the polymonials is {l}')

# 4. Pick some random field element as secret parameter 𝜏, and pretend we have discard it :p
t = F_p.random_element()
print(f'secret parameter 𝜏 is {t}')

# 5. Compute pp(public parameters)
def compute_public_parameters(F_p, t, l):
    
    PP = []
    accumulated = 1
    for i in range(l+1):
        PP.append(F_p.prod((accumulated, g)))
        accumulated = F_p.prod((accumulated, t))
    return PP

PP = compute_public_parameters(F_p, t, l)
print(f'the first 10 elements of public parameters are: {PP[:10]}')
        
# *define a pairing e: G*G-> G_T, we define the simple pairing e(g1, g2) -> g1*g2 (mod p)
def e(g1, g2):
    return F_p.prod((g1, g2))

generator g is 2
the maximum degree l of the polymonials is 6348
secret parameter 𝜏 is 23812
the first 10 elements of public parameters are: [2, 47624, 21525, 18226, 31263, 28231, 7032, 27803, 42760, 2415]


## Step 2: Commit to Polynomials
In reality, we arithmetize circuits and use Plonkish to get polynomials in this step
1. Given a polynomial $\Phi(x) = \sum_{i=0}^l \phi_i x^i$
2. Compute and output commitment $c = g^{\Phi(\tau)}$
   - Wait! $\tau$ has already been discard right? How can committer compute $\tau$?
   - Although he cannot compute $\Phi(\tau)$ directly, he can use public parameters to help with it: 
$$\prod_{i=0}^l(g^{\tau^i})^{\phi_i} = g^{\sum_{i=0}^l \phi_i \tau^i} = g^{\Phi(\tau)}$$

In [17]:
# assume we have done plonk and get our polynomial Φ(𝑥)=4x^2+5x+3 and some point (0,3),(1,12),(3,54) on the poly
# poly_coefs = [3, 5, 4]
poly_point = [(0,3),(1,12),(3,54)]

In [18]:
# create a polynomial ring F_p(x)
F_p_x.<x> = PolynomialRing(F_p)
F_p_x

Univariate Polynomial Ring in x over Finite Field of size 53267

In [19]:
poly = F_p_x.lagrange_polynomial(poly_point)
poly

4*x^2 + 5*x + 3

In [20]:
def poly_commitment(PP, poly):

    com = 1
    poly_coefs = poly.coefficients()
    for i in range(len(poly_coefs)):
        com = G.prod((com,G(PP[i]**poly_coefs[i])))
#         print('PP[',i,']=',PP[i],',poly_coefs[',i,']=',poly_coefs[i],'temp com=',com,'\n')
    return com
        
com = poly_commitment(PP, poly)
print(f'commitment of polynomial is {com}')

commitment of polynomial is 51811


## Step 3: Prove an Evaluation
In this period, the Verifier will ask Prover to 'OPEN' the commitment $c$ to a random specific point $a \in F_p$, in other word, Prover have to evaluation $\Phi(x)$ and commit the result in the form of opening triplet $OT = (a, b, \pi)$
1. Given an evaluation $\Phi(a) = b$
2. Compute and output proof of the evaluation $\pi = g^{q(\tau)}$, where $q(x) := \frac{\Phi(x)-b}{x-a}$
    - $q(x)$ is quotient polynomial: if $\Phi(a) = b$, that means $a$ is a root of $\Phi(x) - b$
    - so $\Phi(x) - b$ can be expressed as $\Phi(x) - b = q(x)(x-a)$, $q(x)$ is a polynomial
    - on the other hand, $q(x)$ exists if and only if $\Phi(a) = b$, so the existence of this quotient polynomial therefore serves as a proof of the evaluation

In [21]:
# Verifier first choose the random point a
a = F_p.random_element()
b = poly(a)

print(f'random element we chosen is {a}, the evaluation b is {b}')

# compute q(x)
q_poly = ((poly-b)/(x-a)).numerator()
print(f'quotient polynomial is {q_poly}')

# compute proof of the evaluation pi
pi = poly_commitment(PP, q_poly)
print(f'proof of the evaluation pi is {pi}')

        

random element we chosen is 52454, the evaluation b is 29731
quotient polynomial is 4*x + 50020
proof of the evaluation pi is 40370


## Step 4: Verify an Evaluation Proof
1. Given a commitment $c = g^{\Phi(\tau)}$, and an evaluation $\Phi(a) = b$, and a proof $\pi = g^{q(\tau)}$
2. Verify that $e(\frac{c}{g^b}, g) = e(\pi,\frac{g^\tau}{g^a})$, where $e$ is a non-trivial bilinear mapping

In [ ]:
# now it is time to do the varification
e(com/G(g**b), g) == e(pi, G(g**t/g**a))

In [ ]:
 e(pi, g**t/g**a)

In [ ]:
e(com/g**b, g)